In [1]:
!pip install twikit

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 2.2 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━

In [ ]:
import asyncio
from twikit import Client

USERNAME = 'username_twitter'
EMAIL = 'email_twitter'
PASSWORD = 'password_twitter'

client = Client("en-US")

async def main():
    await client.login(
        auth_info_1=USERNAME,
        auth_info_2=EMAIL,
        password=PASSWORD,
        cookies_file="cookies.json"
    )
    print("Login OK & cookies saved.")

await main()

Login OK & cookies saved.


In [ ]:
import csv, os, json, asyncio
from datetime import datetime, timezone
from twikit import Client, Forbidden, TooManyRequests, NotFound

CSV_PATH = "tweets_python.csv"
COOKIES  = "cookies.json"

client = Client("en-US")

def _tweet_url(t):
    uname = getattr(t.user, "screen_name", None) or getattr(t.user, "username", None)
    tid   = getattr(t, "id", None) or getattr(t, "rest_id", None)
    return f"https://x.com/{uname}/status/{tid}" if (uname and tid) else ""

def _clean_text(s):
    return (s or "").replace("\n", " ").replace("\r", " ").strip()

async def save_search_paged(query="python", product="Latest", per_page=50, max_rows=500, out_path="tweets_python_paged.csv"):
    client.load_cookies(COOKIES)

    try:
        page = await client.search_tweet(query, product, count=per_page)
    except NotFound:
        if product.lower() == "latest":
            print("'Latest' 404 → fallback ke 'Top'")
            page = await client.search_tweet(query, "Top", count=per_page)
        else:
            raise

    written = 0
    with open(out_path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["name", "username", "text", "created_at_utc", "url"])

        while True:
            batch = list(page)
            if not batch:
                break

            for t in batch:
                uname = getattr(t.user, "screen_name", None) or getattr(t.user, "username", None)
                w.writerow([
                    _clean_text(t.user.name),
                    uname or "",
                    _clean_text(t.text),
                    getattr(t, "created_at", ""),
                    _tweet_url(t),
                ])
                written += 1
                if written >= max_rows:
                    print(f"Saved {written} rows (cap reached) → {os.path.abspath(out_path)}")
                    return

            # coba next page
            try:
                page = await page.next()
            except TooManyRequests:
                print("Rate limit. Tidur 15 detik…")
                await asyncio.sleep(15)
                try:
                    page = await page.next()
                except Exception:
                    break
            except Exception:
                break

            await asyncio.sleep(2)  # jeda sopan

    print(f"Saved {written} rows → {os.path.abspath(out_path)}")

query = "<isi sama kata kunci>"
await save_search_paged(query=query, product="Latest", per_page=50, max_rows=500)


✅ Saved 500 rows (cap reached) → /Users/tiarasabrina/Documents/PROJECT/scrape_twitter/tweets_python_paged.csv
